In [107]:
import matplotlib.pyplot as plt
from matplotlib import animation

import numpy as np
import pandas as pd
import datetime
import random
import time

from string import ascii_lowercase

In [ ]:
# Simple ascii to number dictionary

nodeDict = {}
i = 0
for c in ascii_lowercase:
    nodeDict[c] = i
    i += 1

In [305]:
# Node class
class Node:
    
    def __init__(self, x_pos, node_id, isEvil=False):
        self.chain = []
        self.x_chain = []
        self.x_chain.append(x_pos)
        self.id = node_id
        self.radius = 1
        self.isEvil = isEvil
        
    def connect(self, nodes, timestamp):
        self.chain.append([])
        
        for node in nodes:
            
            # Distance from other node to this node
            # Note that distance (in x_chain is always timestamp + 1 to avoid off-by-one errors)
            dist = (node.x_chain[timestamp] - self.x_chain[timestamp])
            
            # Check if nearby nodes are close enough
            if  abs(dist) <= self.radius and node.id != self.id:
                
                # If the nodes are to this node's right, add them with (self, node)
                if dist > 0:
                    self.chain[timestamp].append([self.id, node.id])

                # Otherwise, add in the order (node, self)
                else:
                    self.chain[timestamp].append([node.id, self.id])
    
    def move(self):
        self.x_pos = self.x_pos + random.randint(-1, 1)

    def __str__(self):
        return ('ID:%3s Position:%3s Evil:%3s' % (self.id, self.x_chain, self.isEvil))

    
    
# EvilNode class
class EvilNode(Node):
    
    def __init__(self, x_pos, node_id):
        super().__init__(self, x_pos, node_id, isEvil=True)
        
    def connect(self, nodes, timestamp):
        self.chain.append([[],[]])
        for node in nodes:

            # Distance from other node to this node
            dist = (node.x_pos - self.x_pos)

            # Check if nearby nodes are close enough
            # Also checks to see if the other Nodes are Evil
            if  abs(dist) <= self.radius and node.id != self.id and node.isEvil == True:

                # If the nodes are to this node's right, add them with (self, node)
                if dist > 0:
                    self.chain[timestamp][1].append([node.id])

                # Otherwise, add in the order (node, self)
                else:
                    self.chain[timestamp][0].append([node.id])

In [306]:
nodeList = []

for c in ascii_lowercase:
    
    # All honest Nodes for now
    if ((random.randint(2,2) % 2) == 0):
        
        node1 = Node(random.randint(0, 50), c)
        nodeList.append(node1)
    else:
        node1 = EvilNode(random.randint(0, 50), c)
        nodeList.append(node1)

In [307]:
for node in nodeList:
    node.connect(nodeList, 0)
    
for node in nodeList:
    print(node)

ID:  a Position:[36] Evil:False
ID:  b Position:[50] Evil:False
ID:  c Position:[11] Evil:False
ID:  d Position:[15] Evil:False
ID:  e Position:[2] Evil:False
ID:  f Position:[7] Evil:False
ID:  g Position:[35] Evil:False
ID:  h Position:[38] Evil:False
ID:  i Position:[8] Evil:False
ID:  j Position:[25] Evil:False
ID:  k Position:[44] Evil:False
ID:  l Position:[10] Evil:False
ID:  m Position:[21] Evil:False
ID:  n Position:[12] Evil:False
ID:  o Position:[35] Evil:False
ID:  p Position:[4] Evil:False
ID:  q Position:[28] Evil:False
ID:  r Position:[22] Evil:False
ID:  s Position:[37] Evil:False
ID:  t Position:[37] Evil:False
ID:  u Position:[49] Evil:False
ID:  v Position:[27] Evil:False
ID:  w Position:[11] Evil:False
ID:  x Position:[45] Evil:False
ID:  y Position:[12] Evil:False
ID:  z Position:[40] Evil:False


In [351]:
# Assumes that nodes accurately report their own location
# Checks all the neighbors to the left and right that the node reports
# Checks all that the left neighbors report each other as neighbors (and also report the node in question)
# Repeat for the right
# 
def queryNode(time, node_id, location, nodeList):
    
    # Grab the appropriate node
    node = nodeList[nodeDict[node_id]]
    
    # Check to see that the node reports it is where it is at
    if (node.x_chain[time] != location):
        return False
    
    else:
        total_witnesses_count = len(node.chain[time])
        unseen_witnesses_count = 0
        unseen_neightbors_count = 0
        
        left_nodes = []
        right_nodes = []
        
        print(left_nodes, right_nodes)
        
        for link in node.chain[time]:
            if (link[0] == node_id):
                right_nodes.append(link[1])
            else:
                left_nodes.append(link[0])
    
        # Check to see if all nodes saw our node
        for neighbor in left_nodes+right_nodes:
            unseen = True
            for link in nodeList[nodeDict[neighbor]].chain[time]:
                if node_id in link:
                    unseen = False
                    print(link)
            if (unseen):
                unseen_witnesses_count += 1
        
        # Checks if all left nodes saw each other (this can blow up quickly in time)
        for neighbor in left_nodes:
            unseen = True
            for link in nodeList[nodeDict[neighbor]].chain[time]:
                for other_neighbors in left_nodes:
                    if other_neighbors != neighbor:
                        if node_id in link:
                            unseen = False
                            print(link)
            if (unseen):
                unseen_neighbors_count += 1
                
        # Checks if all right nodes saw each other (this can blow up quickly in time)
        for neighbor in right_nodes:
            unseen = True
            
            # Iterate through all links per neighbor to see if they saw all other neighbors
            for link in nodeList[nodeDict[neighbor]].chain[time]:
                
                # For each neighbor, check that the link contains them
                for other_neighbors in right_nodes:
                    if other_neighbors != neighbor:
                        if node_id in link:
                            unseen = False
                            print(link)
            if (unseen):
                print(link)
                unseen_neighbors_count += 1
        
        return True, unseen_witnesses_count, unseen_neightbors_count

In [352]:
queryNode(0, 'a', 36, nodeList)

[] []
['g', 'a']
['o', 'a']
['a', 's']
['a', 't']
['g', 'a']
['o', 'a']
['a', 's']
['a', 't']


(True, 0, 0)